# Fetching Pre and Post synaptic Neuron Location

## Libraries

In [1]:
! pip install standard-transform
! pip install --upgrade caveclient

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from caveclient import CAVEclient
from microns_phase3 import nda
from standard_transform import minnie_transform_vx

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


## Preparation

Function that transforms the voxel coorindate into a depth coordinate (function given to us by the authors of the Microns Project)

In [3]:
tform_vx = minnie_transform_vx()

### Querying the data from CAVEclient

In [4]:
client = CAVEclient()

In [5]:
client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

We will be using the latest public release of the dataset, i.e. version 661

In [6]:
client = CAVEclient('minnie65_public')

In [7]:
client.materialize.version = 661

In [8]:
client.materialize.get_tables()

['synapses_pni_2',
 'baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117',
 'coregistration_manual_v3']

These are all the available tables, but we are interested in the *coregistration_manual_v3*, that is the one containing the information about the manually proofread neurons and in the *synapses_pni_2*, which contains all the traced synapses.

In [9]:
df = client.materialize.query_table('coregistration_manual_v3')

We start by setting as the index he ID associated to the root of the synapse

In [10]:
df = df.set_index('pt_root_id')

In [11]:
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
pt_root_id,,,,,,,,,,,,,,,,,,
864691135738685297,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
864691135614842827,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
864691135502985397,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
864691135387371905,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
864691135661410544,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"


In [12]:
df.shape

(13925, 18)

In [13]:
unit_keys_1 = df.iloc[:5000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_2 = df.iloc[5000:10000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_3 = df.iloc[10000:13925, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')

In [14]:
temp_1 = (nda.AreaMembership & unit_keys_1).fetch('brain_area')
print('1/3 Completed!')
temp_2 = (nda.AreaMembership & unit_keys_2).fetch('brain_area')
print('2/3 Completed!')
temp_3 = (nda.AreaMembership & unit_keys_3).fetch('brain_area')
print('3/3 Completed!')
area_membership = np.concatenate((temp_1, temp_2, temp_3), axis=0)
assert len(area_membership) == df.shape[0]

1/3 Completed!
2/3 Completed!
3/3 Completed!


In [15]:
df['Area Membership'] = area_membership
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position,Area Membership
pt_root_id,,,,,,,,,,,,,,,,,,,
864691135738685297,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135614842827,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135502985397,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135387371905,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135661410544,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]",V1


In [16]:
root_ids = df.index.tolist()
len(root_ids)

13925

In [29]:
subset_ids = root_ids[11000:12000]

In [35]:
client.materialize.synapse_query(post_ids=864691135498559507)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,379302380,2020-11-04 06:48:59.510128+00:00,NaN,t,952,107114355401123206,864691136578398228,107114355401112857,864691135498559507,"[308162, 158522, 25488]","[308202, 158582, 25483]","[308158, 158558, 25486]"
1,378825594,2020-11-04 06:48:59.876707+00:00,NaN,t,10712,107403389587061547,864691135990475292,107403389587051129,864691135498559507,"[309854, 215046, 25697]","[309962, 215152, 25698]","[309898, 215090, 25692]"
2,393229390,2020-11-04 06:49:11.546047+00:00,NaN,t,1900,108318114542104122,864691135738307732,108318183261595835,864691135498559507,"[316536, 214654, 25928]","[316544, 214728, 25942]","[316569, 214688, 25937]"
3,360846787,2020-11-04 06:49:08.606602+00:00,NaN,t,968,105715226720528575,864691136689105500,105715226720533369,864691135498559507,"[297690, 220020, 24424]","[297644, 219920, 24430]","[297666, 219998, 24425]"
4,370668670,2020-11-04 06:49:11.336675+00:00,NaN,t,328,106624454318256219,864691135593186456,106624454318253431,864691135498559507,"[304292, 178474, 25716]","[304382, 178426, 25709]","[304348, 178460, 25715]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3266,373037169,2020-11-04 07:09:27.317908+00:00,NaN,t,13576,106909983811195070,864691135571820525,106910052530673062,864691135498559507,"[306560, 209016, 26216]","[306574, 209104, 26213]","[306536, 209047, 26218]"
3267,341428617,2020-11-04 11:08:31.263657+00:00,NaN,t,6012,103671853012726320,864691136423188527,103671784293248816,864691135498559507,"[282940, 199880, 23974]","[282924, 199858, 23968]","[282924, 199870, 23973]"
3268,386507102,2020-11-04 07:09:21.124041+00:00,NaN,t,392,107825601985343568,864691135571820525,107825601985357005,864691135498559507,"[312900, 214934, 25486]","[312938, 215012, 25492]","[312872, 214940, 25487]"
3269,379508055,2020-11-04 08:27:08.909991+00:00,NaN,t,5812,107180532256994709,864691136100848501,107180600976441218,864691135498559507,"[308390, 127682, 25270]","[308476, 127704, 25261]","[308376, 127662, 25267]"


In [34]:
input_values = []
output_values = []
v_area = []

for i in subset_ids:
    print(i)
    input_df = client.materialize.synapse_query(post_ids=i)
    
    input_df.rename(columns = {'pre_pt_root_id':'pt_root_id'}, inplace = True) 
   
    df_matched = pd.merge(input_df,df,on='pt_root_id',how='inner')
    if df_matched.empty:
        continue
        
    #This is the correct depth value that is retrieved by a special function created by the MICrONS team    
    input_vertices_list = tform_vx.apply(df_matched['pt_position'])
    graph_x_stack = np.vstack(input_vertices_list)
    graph_x_values = graph_x_stack[:,1]

    output_vertices = tform_vx.apply(df.at[i,'pt_position'])
    graph_y_value = output_vertices[1]
    if type(graph_y_value) != np.float64:
        continue
    
    num_points = len(graph_x_values)
    input_values.extend(graph_x_values)
    output_values.extend([graph_y_value]*num_points)
    v_area.extend(df_matched['Area Membership'])

864691135346230687
864691135762955830
864691136452161023
864691135395204853
864691136116176548
864691135182207234
864691135408025289
864691135610403079
864691135976435523
864691135275160549
864691137054600566
864691135616500329
864691136051711731
864691136578257428
864691136032680891
864691135065343812
864691136100720245
864691135565539863
864691135927333332
864691136092024756
864691136092024756
864691135954616584
864691135478796230
864691135269746853
864691137197833793
864691135976518467
864691135123255079
864691135468907090
864691135841553635
864691135841553635
864691135416231866
864691135915709030
864691135429541936
864691135463347518
864691135915507558
864691135278563489
864691136008490412
864691135701135227
864691135701135227
864691135749482793
864691136106267609
864691135615032779
864691136391331967
864691135104954189
864691135740500116
864691135874000526
864691135468879186
864691135468879186
864691136195208424
864691135645460719
864691135472967474
864691135374478153
864691136144

HTTPError: 500 Server Error: error with status PGRES_COPY_OUT and no message from the libpq for url: https://minnie.microns-daf.com/materialize/api/v3/datastack/minnie65_public/version/661/table/synapses_pni_2/query?return_pyarrow=True&arrow_format=True&split_positions=True content:b'{"code": 500, "message": "error with status PGRES_COPY_OUT and no message from the libpq", "traceback": ["Traceback (most recent call last):\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask/app.py\\", line 1516, in full_dispatch_request\\n    rv = self.dispatch_request()\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask/app.py\\", line 1502, in dispatch_request\\n    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask_restx/api.py\\", line 403, in wrapper\\n    resp = resource(*args, **kwargs)\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask/views.py\\", line 84, in view\\n    return cur

In [ ]:
df_export = pd.DataFrame(list(zip(input_values, output_values, v_area)), columns=['Inputh Depth', 'Output Depth', 'Area Membership'])
df_export.head()

,Inputh Depth,Output Depth,Area Membership
0,335.410876,349.834836,RL
1,295.587597,349.834836,V1
2,309.880951,349.834836,RL
3,344.816976,349.834836,V1
4,403.543122,349.834836,RL


In [ ]:
df_export.to_csv('graph_values_12.csv')